### 7.  Appendix

**Python code**

In [ ]:
import matplotlib 
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns 
sns.set(context="notebook", palette="Spectral", style = 'darkgrid' ,font_scale = 1.5, color_codes=True)
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import model_selection
from sklearn.linear_model import RidgeCV, LassoCV,ElasticNetCV
from sklearn.model_selection import RepeatedKFold
from numpy import arange
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
from scipy.stats import shapiro
from scipy import stats
import plotnine as p9
from mizani.breaks import date_breaks
from mizani.formatters import date_format
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

**4.1 Data cleaning**

In [ ]:
#Import data
df = pd.read_csv('data/train.csv')

4.1.1 Missing value

*Get the percentage of NA values for each column*

In [ ]:
na_freq = df.apply(lambda x: sum(x.isna()) / len(x), axis=0)
na_freq = (
    na_freq[na_freq>0]
    .reset_index()
    .rename(columns={'index': 'Feature', 0: 'Missing %'})
    .sort_values(by='Missing %', ascending=False)
)
na_freq['Feature'] = pd.Categorical(na_freq['Feature'], categories=na_freq['Feature'].tolist())
(
    p9.ggplot(na_freq, p9.aes('Feature', 'Missing %'))
    + p9.geom_bar(stat='identity')
    + p9.scale_y_continuous(labels=lambda x: ["{:.0%}".format(v) for v in x])
    + p9.theme(
        axis_text_x = p9.element_text(angle=35,hjust=1),
        figure_size = (7, 3)
    )
)

In [ ]:
df.columns

In [ ]:
print(sorted(df['Neighborhood'].unique()))

*Find the reason for missing: missing at random, or missing for a reason*

Missing because the component does not exist in the house:
* PoolQC
* MiscFeature
* Alley
* Fence
* FireplaceQu
* GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond
* BsmtExposure (NA and "No" have different meaning)
* BsmtFinType1, BsmtFinType2, BsmtCond, BsmtQual

Missing at random
* LotFrontage
* MasVnrType & MasVnrArea
* Electrical

*Example of missing due to non-existance of a house component*

In [ ]:
# Define two helper functions for calculating percentiles
def q05(x):
    return np.quantile(x, 0.05)
def q95(x):
    return np.quantile(x, 0.95)

In [ ]:
# When there is no pool (PoolArea == 0), PoolQC is missing
(
    df.groupby('PoolQC', dropna=False)
    .agg({
        'PoolArea': [q05, np.median, q95],
        'SalePrice': [q05, np.median, q95]
    })
)

*Confirm missing at random*

In [ ]:
# LotFrontage:
# Missing vs non-missing: similar sale price and year built
#    (although there are some difference in median sale price)
# So we consider this as missing at random
# Consider removing this column
# Since this column is highly correlated with LotArea, we may consider removing this feature.

(
    df.groupby(df['LotFrontage'].isna())
    .agg({
        'LotFrontage': [len, q05, np.median, q95],
        'SalePrice': [q05, np.median, q95],
        'YearBuilt': [q05, np.median, q95],
        'LotArea': [q05, np.median, q95],
        'LotShape': lambda x: x.value_counts()
    })
)

In [ ]:
(
    p9.ggplot(df, p9.aes('LotArea', 'LotFrontage', color='LotShape'))
    + p9.geom_point(alpha=0.5)
    + p9.xlim(0, 20000)
    + p9.ylim(0, 180)
)

In [ ]:
# MasVnrType and MasVnrArea

# When MasVnrType is missing, the corresponding MasVnrArea value is also missing
# No systematic difference in the values in year_built and neighborhood
#    between MasVnrType missing and non-missing groups

# Fill in "None" and "0" for Type and Area, because that is the most observed values

(
    df.groupby('MasVnrType', dropna=False)
    .agg({
        'MasVnrArea': [len, q05, np.median, q95],
        'SalePrice': np.median,
        'YearBuilt': [q05, np.median, q95],
        'Neighborhood': lambda x: x.unique()
    })
)

In [ ]:
(
    df.groupby(df['MasVnrType'].isna())
    .agg({
        'MasVnrArea': [len, q05, np.median, q95],
        'SalePrice': np.median,
        'YearBuilt': [q05, np.median, q95],
        'Neighborhood': lambda x: x.unique()
    })
)

In [ ]:
# Electrical

# There is only one missing data for the column "Electrical"
# And the "Utilities" column says it has all utilities.

# Since only one row, it's safe to remove this data

df[df['Electrical'].isna()]['Utilities']

*Pre-process the missing values*

In [ ]:
df.columns

In [ ]:
categorical_feats = [
    'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu',
    'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
    'BsmtExposure', 'BsmtCond', 'BsmtQual'
]
# 'GarageYrBlt'

In [ ]:
df['GarageYrBlt'].fillna(0, inplace=True)

In [ ]:
df[categorical_feats] = df[categorical_feats].fillna('Not exist')

In [ ]:
sel = df['BsmtFinType1'].isna() & df['BsmtFinType2'].isna()
df.loc[sel, 'BsmtFinType1'] = 'Not exist'
df.loc[sel, 'BsmtFinType2'] = 'Not exist'
df.loc[df['BsmtFinType2'].isna(), 'BsmtFinType2'] = 'Unf'

In [ ]:
df = df.drop(columns='LotFrontage')

In [ ]:
df.loc[df['MasVnrType'].isna(), 'MasVnrType'] = 'None'

In [ ]:
df.loc[df['MasVnrArea'].isna(), 'MasVnrArea'] = 0

In [ ]:
# After process missing values
msno.matrix(df)

In [ ]:
df.columns

**4.1.2 Outliers**

In [ ]:
# Training data and testing data
# x and y
x = df.drop(["logSalePrice","SalePrice"],axis=1) # create x variables, drop the independent variable
y = df.logSalePrice

x_train, x_test, y_train, y_test = train_test_split(x, y,random_state = 0,test_size=0.25)

#log transform SalePrice
x = df.SalePrice
sns.set_style('whitegrid')
sns.distplot(x)
plt.show()

df['SalePrice_log'] = np.log(df.SalePrice)
x = df.SalePrice_log
sns.distplot(x)
plt.show()

In [ ]:
#OLS model
model = sm.OLS(y, x, missing='drop')
model_result = model.fit()

In [ ]:
#Outlying in X
# normalized residuals
model_norm_residuals = model_result.get_influence().resid_studentized_internal
# leverage, from statsmodels internals
model_leverage = model_result.get_influence().hat_matrix_diag
# cook's distance, from statsmodels internals
model_cooks = model_result.get_influence().cooks_distance[0]

# Residuals and leverage plot
plot_lm_4 = plt.figure();
plt.scatter(model_leverage, model_norm_residuals, alpha=0.5);
sns.regplot(model_leverage, model_norm_residuals,
              scatter=False,
              ci=False,
              lowess=True,
              line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8});
plot_lm_4.axes[0].set_xlim(0, max(model_leverage)+0.01)
plot_lm_4.axes[0].set_ylim(-8, 8)
plot_lm_4.axes[0].set_title('Residuals vs Leverage')
plot_lm_4.axes[0].set_xlabel('Leverage')
plot_lm_4.axes[0].set_ylabel('Standardized Residuals');

# annotations
leverage_top_3 = np.flip(np.argsort(model_cooks), 0)[:3]
for i in leverage_top_3:
    plot_lm_4.axes[0].annotate(i,
                        xy=(model_leverage[i],
                        model_norm_residuals[i]));
    

In [ ]:
#Outlying in y
#At alpha = 0.05, Single tail
t= stats.t.ppf(1-0.05/(2 * Train.shape[0]), Train.shape[0]-Train.shape[1])

p = sns.scatterplot(y_pred,residuals)
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.ylim(-5,5)
plt.xlim(10,14)
p = sns.lineplot([10,14],[0,0],color='blue')
p = plt.title('Residuals vs fitted values')
p = sns.lineplot([10,14],[t,t],color='green')
p = sns.lineplot([10,14],[-t,-t],color='green')

**4.2 Assumptions checks**

In [ ]:
# Linearity
p = sns.scatterplot(y_pred,residuals)
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.ylim(-0.5,0.5)
plt.xlim(10,14)
p = sns.lineplot([10,14],[0,0],color='blue')
p = plt.title('Residuals vs fitted values')

# Normality of error terms: QQ plot
sm.qqplot(model_result.resid, line='s');
p = plt.title('QQ Plot of residuals')

# Independence of the error terms:
import statsmodels.api as sm
# autocorrelation
sm.graphics.tsa.plot_acf(residuals, lags=40)
plt.title('Residuals autocorrelation')
plt.show()

#Constant variance of the error terms / homoscedastic:
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
name = ['F statistic', 'p-value']
test = sms.het_goldfeldquandt(residuals, x_train)
lzip(name, test)

#Multicollinearity (Correlation)
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = x.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(x.values, i)
                          for i in range(len(x.columns))]
  
#print(vif_data)
VIF_sort= vif_data.sort_values(by='VIF',ascending=False)

**4.3 Feature Engineering**


4.3.1 Categorical variable encoding

In [ ]:
(
    df.groupby(['YrSold', 'Neighborhood'])
    .agg({'SalePrice': np.mean})
    .reset_index()
    .pivot(index='Neighborhood', columns='YrSold')
).style.format('{:.0f}')

In [ ]:
num_to_categorical_feats = ['MSSubClass', 'OverallQual', 'OverallCond', 'YrSold']
for col in num_to_categorical_feats:
    df[col] = df[col].astype(str)

basement_feats = set(['BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2'])
feats_to_combine = set(['Condition1', 'Condition2', 'Exterior1st', 'Exterior2nd'])
one_hot_feats = set(df.dtypes[df.dtypes == 'object'].index) - basement_feats - feats_to_combine

In [ ]:
# basement_feats contains two categorical and two numerical
print(
    'Number of categorical features: ',
    len(one_hot_feats) + len(feats_to_combine) + len(basement_feats) / 2
)

In [ ]:
(sum(df['BsmtFinSF1'] == 0), sum(df['BsmtFinSF2'] == 0))

In [ ]:
# df = pd.get_dummies(df, columns=one_hot_feats, drop_first=True, dummy_na=True)
df = pd.get_dummies(df, columns=one_hot_feats, drop_first=True)

In [ ]:
basement_type = set(df['BsmtFinType1'].unique()).union(set(df['BsmtFinType2'].unique()))
# add columns to df
for bt in basement_type:
    df['BsmtFinSF_' + bt] = 0

# fill in square feet values into the columns
for r in range(len(df)):
    col = 'BsmtFinSF_' + df.loc[r, 'BsmtFinType1']
    if df.loc[r, 'BsmtFinType1'] == 'Not exist':
        df.loc[r, col] = 1
    else:
        df.loc[r, col] = df.loc[r, 'BsmtFinSF1']
    
    # Fill in the number for the second BsmtFinType.
    col = 'BsmtFinSF_' + df.loc[r, 'BsmtFinType2']
    if df.loc[r, 'BsmtFinType2'] == 'Not exist':
        df.loc[r, col] = 1
    else:
        # If the col is the same as the prev col, we just add the numbers together.
        df.loc[r, col] += df.loc[r, 'BsmtFinSF2']

In [ ]:
def combine_features(df, feat1, feat2, feat_prefix):
    feat_levels = set(df[feat1].unique()).union(set(df[feat2].unique()))
    print('Feature levels = ', feat_levels)
    # create new columns for each level of the feature
    # and initialize to all 0's
    for l in feat_levels:
        df[feat_prefix + l] = '0'
    
    # fill in value 1 to the relevant columns
    for r in range(len(df)):
        col = feat_prefix + df.loc[r, feat1]
        df.loc[r, col] = '1'
        col = feat_prefix + df.loc[r, feat2]
        df.loc[r, col] = '1'
        
    return df

In [ ]:
wrong_spelling = {'Wd Shng': 'Wd Sdng', 'CmentBd': 'CemntBd'}
df['Exterior1st'].replace(wrong_spelling, inplace=True)
df['Exterior2nd'].replace(wrong_spelling, inplace=True)

In [ ]:
df = combine_features(df, 'Condition1', 'Condition2', 'Condition_')
df = combine_features(df, 'Exterior1st', 'Exterior2nd', 'Exterior_')

In [ ]:
# Check the feature transformed result
df[['BsmtFinSF_ALQ', 'BsmtFinSF_Rec', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2']].tail(6)

In [ ]:
df['Condition_Feedr'].head(5)

In [ ]:
df[['Condition1', 'Condition2']].iloc[[974, 1003, 88]]

In [ ]:
dummy_example_df = pd.get_dummies(
    df[['Condition1', 'Condition2']].iloc[[974, 1003, 88]])

dummy_example2_df = pd.concat([
    dummy_example_df['Condition1_Feedr'] | dummy_example_df['Condition2_Feedr'],
    dummy_example_df['Condition1_RRAn'] | dummy_example_df['Condition2_RRAn']
], axis=1).rename(columns={0: 'Condition_Feedr', 1: 'Condition_RRAn'})

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(11,3), gridspec_kw={'width_ratios': [2, 1]})

sns.heatmap(dummy_example_df, annot=True, cbar=False, ax=ax[0])
sns.heatmap(dummy_example2_df, annot=True, cbar=False, ax=ax[1])

for i in range(2):
    ax[i].set_ylabel('Sample ID')
    ax[i].set_xlabel('Feature Name')
    # ax[i].xticks(rotation=10)

ax[0].set_title('(a) One-hot Encoding')
ax[1].set_title('(b) Two-hot Encoding')

4.3.2 Nonlinearity transformation

In [ ]:
df['MoSold_sq'] = df['MoSold'] ** 2

In [ ]:
df['YearBuilt_sq'] = (df['YearBuilt'] - 1850) ** 2

In [ ]:
pd.DataFrame({'col': df.columns, 'type': list(df.dtypes)}).head(5)

In [ ]:
# YearBuilt: nonlinear relation
(
    p9.ggplot(df, p9.aes(x='YearBuilt', y='SalePrice'))
    + p9.geom_point(alpha=0.4)
    + p9.scale_y_log10()
    + p9.geom_smooth(method='loess', color='tab:blue')
    + p9.theme(figure_size=(6, 3))
)

In [ ]:
# MoSold: nonlinear relation
(
    p9.ggplot(df, p9.aes(x='MoSold', y='SalePrice', group='MoSold'))
    + p9.geom_boxplot()
    + p9.scale_y_log10()
    + p9.scale_x_discrete(limits=np.arange(1,13))
    # + p9.geom_smooth(method='loess', color='tab:blue')
    + p9.theme(figure_size=(7, 4)) + p9.facet_wrap('~ YrSold')
)

4.3.3 Interaction

In [ ]:
df['BedroomAbvGr*GarageCars'] = df['BedroomAbvGr'] * df['GarageCars']
df['BedroomAbvGr*FullBath'] = df['BedroomAbvGr'] * df['FullBath']
df['GrLivArea*LotArea'] = df['GrLivArea'] * df['LotArea']

**Building models**

Elastic Net Regression

In [ ]:
import matplotlib 
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import model_selection
from sklearn.linear_model import RidgeCV, LassoCV,ElasticNetCV
from sklearn.model_selection import RepeatedKFold
from numpy import arange
from sklearn.model_selection import GridSearchCV

In [ ]:
Train = pd.read_csv('train_processed.csv')
print(Train.columns)

In [ ]:
feats = list(set(Train.columns) - set(['logSalePrice', 'SalePrice']))
df_normalized = Train.copy()
df_normalized[feats] = (Train[feats] - Train[feats].mean()) / Train[feats].std()
# df_normalized['logSalePrice'] = df['logSalePrice']
df_normalized['SalePrice'] = Train['SalePrice'] / 1000000  # convert the unit to millions

In [ ]:
def train_and_predict(
    df, feats, target,
    alphas = [.1, .5, .7, .9, .95, .99, 1], is_log_transformed = True,
    random_state=1000
):
    
    # Train-test split. Use 30% as testing data
    X_train, X_test, y_train, y_test = train_test_split(
        df[feats], df[target], test_size=0.3, random_state=random_state)

    # Elastic Net cross-validation
    model = ElasticNetCV(cv = 10,alphas = alphas,random_state = random_state, n_jobs = -1)
    model.fit(X_train, y_train)

    # Create a data frame to store how MSE changes with alpha (regularization strength)
    mse_path_df = pd.DataFrame({
        'alpha': model.alphas_,
        'mse': np.mean(model.mse_path_, axis=1)
    })

    # Predict on test data
    pred_test = model.predict(X_test)
    
    # If log transform is used, we transform the data back to the original unit
    if is_log_transformed:
        pred_test = np.exp(pred_test) / 1000000  # in million dollar
        y_test = np.exp(y_test) / 1000000  # in million dollar
    
    # Calculate RMSE and MAPE
    rmse = np.sqrt(np.mean(np.square(pred_test - y_test)))
    mape = np.mean(np.abs(pred_test - y_test) / y_test)
    
    return {
        'mse_path': mse_path_df,
        'pred_test': pred_test,
        'X_test': X_test,  # return X_test for debugging purpose
        'y_test': y_test,
        'rmse': rmse,
        'mape': mape,
        'elasticnet_cv': model  # store the whole elasticnet_cv object for easier exploration later
    }

In [ ]:
alphas= np.linspace(0.001**3,0.005,500)
#tcopy = Train.copy()
res = train_and_predict(df_normalized, feats,'logSalePrice',alphas = alphas,is_log_transformed = True)

In [ ]:
sum(res['elasticnet_cv'].coef_ != 0)

In [ ]:
res['elasticnet_cv'].alpha_

In [ ]:
def plot_results(res):
    plt.figure(figsize=(15, 10))
    
    # Scatter plot of the prediction
    plt.subplot(221)
    plt.plot(res['pred_test'], res['y_test'], '.')
    plt.xlabel('Predicted Sale Price')
    plt.ylabel('True Sale Price')
    
    # use get_xlim for both x and y to draw "x=y" line
    plt.plot(plt.gca().get_xlim(), plt.gca().get_xlim())
    
    # Residual plot
    plt.subplot(222)
    plt.plot(res['pred_test'], res['y_test'] - res['pred_test'], '.')
    plt.xlabel('Predicted Sale Price')
    plt.ylabel('Residual')
    
    # plot a horizontal line
    plt.plot(plt.gca().get_xlim(), [0, 0])

    # LASSO: mse vs penalty strength
    plt.subplot(212)
    plt.plot(res['mse_path']['alpha'], res['mse_path']['mse'], '.-')
    plt.xlabel('Penalty Strength')
    plt.ylabel('MSE')
    plt.grid()
    print('Test RMSE: {:.3f} million, or {:.2f}k'.format(res['rmse'], res['rmse'] * 1000))
    print('Test MAPE: {:.2%}'.format(res['mape']))

In [ ]:
plot_results(res)

Final Model Using Optimized Alpha

In [ ]:
# split data into training, testing
X_train, X_test, y_train, y_test = train_test_split(
        df_normalized[feats], df_normalized['logSalePrice'], test_size=0.3, random_state=1000)
# elastic net using optimized alpha
model = ElasticNet(alpha=res['elasticnet_cv'].alpha_, l1_ratio=0.5)
model.fit(X_train,y_train)
# predict data using elastic net model
y_pred = model.predict(X_test)
print ("Test MSE - Elastic Net Regression:",((mean_squared_error(y_test, y_pred ))))
print ("Elastic Net Score (R squared) :",((ElasticNet.score(model, X_test,y_test))))

In [ ]:
# set L to be number of coefficients
L = len(res['elasticnet_cv'].coef_)
nonzero_idx = []
coefs = []
for i in range(L):
    if res['elasticnet_cv'].coef_[i] != 0:
        # append index i if coef is 0
        nonzero_idx = np.append(nonzero_idx,i)
        coefs = np.append(coefs,res['elasticnet_cv'].coef_[i])
# find column names    
tcolnames = []       
for col in df_normalized[feats].columns:
    tcolnames = np.append(tcolnames,col)

In [ ]:
nz = nonzero_idx.shape[0]
nz_colnames = []
for i in range(nz):
    # append non-zero coefficient names
    nz_colnames = np.append(nz_colnames,tcolnames[int(nonzero_idx[i])])

LASSO regression

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotnine as p9
from sklearn.linear_model import Lasso, LassoCV, lasso_path, enet_path, Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

In [ ]:
df = pd.read_csv('data/train_processed.csv')

In [ ]:
# Normalize the features

feats = list(set(df.columns) - set(['logSalePrice', 'SalePrice']))
df_normalized = df.copy()
df_normalized[feats] = (df[feats] - df[feats].mean()) / df[feats].std()
# df_normalized['logSalePrice'] = df['logSalePrice']
df_normalized['SalePrice'] = df['SalePrice'] / 1000000  # convert the unit to millions

In [ ]:
X = df_normalized[feats]
y = df_normalized['logSalePrice']

alphas = 10 ** np.arange(-3.5, 2, 0.01)

In [ ]:
alphas_lasso, coefs_lasso, _ = lasso_path(X, y, alphas=alphas)
alphas_enet, coefs_enet, _ = enet_path(
    X, y, l1_ratio=0.2,
    alphas=alphas)

In [ ]:
np.sum(np.abs(coefs_lasso[:, 0]))

Ridge regression

In [ ]:
alphas_ridge = 10 ** np.arange(-3.5, 2, 0.01)
clf = Ridge()
coefs_ridge = np.empty((X.shape[1], len(alphas_ridge)))
for i in range(len(alphas_ridge)):
    clf.set_params(alpha=alphas_ridge[i])
    clf.fit(X, y)
    coefs_ridge[:, i] = list(clf.coef_)

Barplot for Model Comparison

In [ ]:
import seaborn as sns

In [ ]:
model_comparison = pd.DataFrame({
    'RMSE': np.array([31.8,30.83,32.68]),
    'MAPE %': np.array([10.19,9.61,9.73]),
    'Model': ['Ridge','ElasticNet','LASSO']
})

Model_comparison

In [ ]:
#fig, ax = plt.subplots()
sns.set(style="ticks")
ax = sns.barplot(x="Model", y="RMSE", data = model_comparison)
ax.set_ylim(30,35)
ax.bar_label(ax.containers[0])

In [ ]:
#fig, ax = plt.subplots()
sns.set(style="ticks")
ax = sns.barplot(x="Model", y="MAPE %", data = model_comparison)
ax.set_ylim(9,11)
ax.bar_label(ax.containers[0])

**5.2 Coefficient Path Analysis**

In [ ]:
plt.figure(figsize=(15, 10))
for i in range(50):
    plt.subplot(5,10,i+1)
    var_id = i
    plt.plot(np.log(alphas_lasso), coefs_lasso[var_id, :])
    plt.plot(np.log(alphas_enet), coefs_enet[var_id, :])
    plt.plot(np.log(alphas_ridge), coefs_ridge[var_id, :])

In [ ]:
var_id = 14
plt.plot(-np.log(alphas_lasso), coefs_lasso[var_id, :], label='LASSO')
plt.plot(-np.log(alphas_enet), coefs_enet[var_id, :], label='ENet')
plt.plot(-np.log(alphas_ridge), coefs_ridge[var_id, :], label='Ridge')
plt.title('Number of Bedroom Above Grade')
plt.legend()

In [ ]:
df.columns[32]

In [ ]:
var_id = 32
plt.plot(alphas_lasso, coefs_lasso[var_id, :], label='LASSO')
plt.plot(alphas_enet, coefs_enet[var_id, :], label='ENet')
plt.plot(alphas_ridge, coefs_ridge[var_id, :], label='Ridge')
plt.title('RoofStyle_Mansard')
plt.gca().set_xscale('log')
plt.gca().invert_xaxis()
plt.legend()

In [ ]:
fig, ax = plt.subplots(3, 1, sharex=True, figsize=(6, 10))
for var_id in range(20):
    ax[0].plot(alphas_lasso, coefs_lasso[var_id, :])
    ax[0].text(alphas_lasso[-1], coefs_lasso[var_id, -1], str(var_id))
    ax[0].set_title('LASSO')
    
for var_id in range(20):
    ax[1].plot(alphas_enet, coefs_enet[var_id, :])
    ax[1].set_title('Elastic Net')

for var_id in range(20):
    ax[2].plot(alphas_ridge, coefs_ridge[var_id, :])
    ax[2].set_title('Ridge')

# Because x axis is shared, change one x axis will affect all subplots
ax[2].set_xscale('log')
ax[2].invert_xaxis()
ax[2].set_xlabel('Regularization Strength ($\\alpha$)')

for i in range(3):
    ax[i].set_ylabel('Coefficient')

# Add lines to show first non-zero coefficient
idx_1st_non_zero = np.where(coefs_lasso[0:20, :].sum(axis=0))[0][0]
alpha_1st_non_zero = alphas_lasso[idx_1st_non_zero]
ax[0].axvline(x=alpha_1st_non_zero, linestyle='--')

idx_1st_non_zero = np.where(coefs_enet[0:20, :].sum(axis=0))[0][0]
alpha_1st_non_zero = alphas_enet[idx_1st_non_zero]
ax[1].axvline(x=alpha_1st_non_zero, linestyle='--')

In [ ]:
def plot_path_horizontal(
    var_id_list, feat_name, ylim=(0, 0.2), legend=True, figsize=(10,4)
):
    fig, ax = plt.subplots(1, 3, sharex=True, sharey=True,
                           figsize=figsize)
    # fig.suptitle('Feature Coefficient Path Comparison for Three Models')
    for var_id in var_id_list:
        ax[0].plot(alphas_lasso, coefs_lasso[var_id, :], label=feat_name[var_id])
        # ax[0].text(alphas_lasso[-1], coefs_lasso[var_id, -1], str(var_id))
        ax[0].set_title('LASSO')
        ax[0].set_ylim(ylim)

    for var_id in var_id_list:
        ax[1].plot(alphas_enet, coefs_enet[var_id, :], label=feat_name[var_id])
        ax[1].set_title('Elastic Net')
        ax[1].set_ylim(ylim)

    for var_id in var_id_list:
        ax[2].plot(alphas_ridge, coefs_ridge[var_id, :], label=feat_name[var_id])
        ax[2].set_title('Ridge')
        ax[2].set_ylim(ylim)

    # Because x axis is shared, change one x axis will affect all subplots
    ax[2].set_xscale('log')
    ax[2].invert_xaxis()
    
    # Set axis labels
    ax[0].set_ylabel('Coefficient')
    for i in range(3):
        ax[i].set_xlabel('Regularization Strength ($\\alpha$)')
        if legend:
            ax[i].legend()

    # Add lines to show first non-zero coefficient
    idx_1st_non_zero = np.where(coefs_lasso[var_id_list, :].sum(axis=0))[0][0]
    alpha_1st_non_zero = alphas_lasso[idx_1st_non_zero]
    ax[0].axvline(x=alpha_1st_non_zero, linestyle='--')

    idx_1st_non_zero = np.where(coefs_enet[var_id_list, :].sum(axis=0))[0][0]
    alpha_1st_non_zero = alphas_enet[idx_1st_non_zero]
    ax[1].axvline(x=alpha_1st_non_zero, linestyle='--')

In [ ]:
def plot_path_vertical(
    var_id_list, feat_name, ylim=(0, 0.2), legend=True, figsize=(6, 10)
):
    
    fig, ax = plt.subplots(3, 1, sharex=True, figsize=figsize)
    fig.suptitle('Feature Coefficient Path')
    for var_id in var_id_list:
        ax[0].plot(alphas_lasso, coefs_lasso[var_id, :], label=feat_name[var_id])
        # ax[0].text(alphas_lasso[-1], coefs_lasso[var_id, -1], str(var_id))
        ax[0].set_title('LASSO')
        ax[0].set_ylim(ylim)

    for var_id in var_id_list:
        ax[1].plot(alphas_enet, coefs_enet[var_id, :], label=feat_name[var_id])
        ax[1].set_title('Elastic Net')
        ax[1].set_ylim(ylim)

    for var_id in var_id_list:
        ax[2].plot(alphas_ridge, coefs_ridge[var_id, :], label=feat_name[var_id])
        ax[2].set_title('Ridge')
        ax[2].set_ylim(ylim)

    # Because x axis is shared, change one x axis will affect all subplots
    ax[2].set_xscale('log')
    ax[2].invert_xaxis()
    ax[2].set_xlabel('Regularization Strength ($\\alpha$)')

    for i in range(3):
        ax[i].set_ylabel('Coefficient')
        if legend:
            ax[i].legend()

    # Add lines to show first non-zero coefficient
    idx_1st_non_zero = np.where(coefs_lasso[var_id_list, :].sum(axis=0))[0][0]
    alpha_1st_non_zero = alphas_lasso[idx_1st_non_zero]
    ax[0].axvline(x=alpha_1st_non_zero, linestyle='--')

    idx_1st_non_zero = np.where(coefs_enet[var_id_list, :].sum(axis=0))[0][0]
    alpha_1st_non_zero = alphas_enet[idx_1st_non_zero]
    ax[1].axvline(x=alpha_1st_non_zero, linestyle='--')

In [ ]:
np.corrcoef(df_normalized['GrLivArea'], df_normalized['1stFlrSF'])

In [ ]:
np.corrcoef(df_normalized['GrLivArea'], df_normalized['2ndFlrSF'])

In [ ]:
np.corrcoef(df_normalized['1stFlrSF'], df_normalized['2ndFlrSF'])

In [ ]:
var_id_list = [129,250,238]
plot_path_horizontal(var_id_list, feats, figsize=(10,3), ylim=(-0.005, 0.2))

In [ ]:
var_id_list = np.arange(20)
plot_path_horizontal(var_id_list, feats, figsize=(10,3), ylim=(-0.1, 0.25), legend=False)

In [ ]:
[(i, X.columns[i])  for i in range(len(X.columns)) if 'Area' in X.columns[i]]

In [ ]:
[(i, X.columns[i])  for i in range(len(X.columns)) if 'SF' in X.columns[i]]

**5.3 Feature Coefficient Stability**

In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import plotnine as p9
from mizani.breaks import date_breaks
from mizani.formatters import date_format

from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

In [ ]:
df = pd.read_csv('data/train_processed.csv')

In [ ]:
sum(df['PoolArea'] > 0) / len(df)

In [ ]:
# Normalize the features

feats = list(set(df.columns) - set(['logSalePrice', 'SalePrice']))
df_normalized = df.copy()
df_normalized[feats] = (df[feats] - df[feats].mean()) / df[feats].std()
# df_normalized['logSalePrice'] = df['logSalePrice']
df_normalized['SalePrice'] = df['SalePrice'] / 1000000  # convert the unit to millions

In [ ]:
def train_and_predict(
    df, feats, target, is_log_transformed=True,
    alphas=10 ** np.arange(-3.5, -2.8, 0.01),
    random_state=1000
):
    
    # Train-test split. Use 30% as testing data
    X_train, X_test, y_train, y_test = train_test_split(
        df[feats], df[target], test_size=0.3, random_state=random_state)

    # Lasso cross-validation
    lasso_cv = LassoCV(cv=10, random_state=random_state, alphas=alphas, n_jobs=-1)
    lasso_cv.fit(X_train, y_train)

    # Create a data frame to store how MSE changes with alpha (regularization strength)
    mse_path_df = pd.DataFrame({
        'alpha': lasso_cv.alphas_,
        'mse': np.mean(lasso_cv.mse_path_, axis=1)
    })

    # Predict on test data
    pred_test = lasso_cv.predict(X_test)
    
    # If log transform is used, we transform the data back to the original unit
    if is_log_transformed:
        pred_test = np.exp(pred_test) / 1000000  # in million dollar
        y_test = np.exp(y_test) / 1000000  # in million dollar
    
    # Calculate RMSE and MAPE
    rmse = np.sqrt(np.mean(np.square(pred_test - y_test)))
    mape = np.mean(np.abs(pred_test - y_test) / y_test)
    
    return {
        'mse_path': mse_path_df,
        'pred_test': pred_test,
        'X_test': X_test,  # return X_test for debugging purpose
        'y_test': y_test,
        'rmse': rmse,
        'mape': mape,
        'lasso_cv': lasso_cv  # store the whole lasso_cv object for easier exploration later
    }

In [ ]:
def plot_results(res):
    plt.figure(figsize=(10, 7))
    
    # Scatter plot of the prediction
    plt.subplot(221)
    plt.plot(res['pred_test'], res['y_test'], '.')
    plt.xlabel('Predicted Sale Price')
    plt.ylabel('True Sale Price')
    
    # use get_xlim for both x and y to draw "x=y" line
    plt.plot(plt.gca().get_xlim(), plt.gca().get_xlim())
    
    # Residual plot
    plt.subplot(222)
    plt.plot(res['pred_test'], res['y_test'] - res['pred_test'], '.')
    plt.xlabel('Predicted Sale Price')
    plt.ylabel('Residual')
    
    # plot a horizontal line
    plt.plot(plt.gca().get_xlim(), [0, 0])

    # LASSO: mse vs penalty strength
    plt.subplot(212)
    plt.plot(res['mse_path']['alpha'], res['mse_path']['mse'], '.-')
    plt.xlabel('Penalty Strength')
    plt.ylabel('MSE')
    plt.grid()
    print('Test RMSE: {:.3f} million, or {:.2f}k'.format(res['rmse'], res['rmse'] * 1000))
    print('Test MAPE: {:.2%}'.format(res['mape']))
    

In [ ]:
feat_coef_df = pd.DataFrame({
    'feat_name': res['lasso_cv'].feature_names_in_,
    'coef': res['lasso_cv'].coef_,
    'abs_coef': np.abs(res['lasso_cv'].coef_)
})

In [ ]:
# Sort the features based on the absolute value of the coefficients
feat_coef_df = feat_coef_df.sort_values(by='abs_coef', ascending=False)

In [ ]:
def add_feat_type(feat_coef_df):
    feat_coef_df['feat_type'] = 'Others'
    feat_coef_df['feat_type'] = np.where(
        feat_coef_df['feat_name'].apply(lambda x: 'Pool' in x),
        'Pool', feat_coef_df['feat_type']
    )
    
    feat_coef_df['feat_type'] = np.where(
        feat_coef_df['feat_name'].apply(lambda x: ('Area' in x and 'Pool' not in x) or ('SF' in x) or ('GarageCars' == x)),
        'Area / Size', feat_coef_df['feat_type']
    )
    
    feat_coef_df['feat_type'] = np.where(
        feat_coef_df['feat_name'].apply(lambda x: 'Zoning' in x or 'Neighborhood' in x),
        'Location', feat_coef_df['feat_type']
    )
    
    return feat_coef_df

In [ ]:
feat_coef_df = add_feat_type(feat_coef_df)

In [ ]:
fig_rank_1 = (
    p9.ggplot(
        feat_coef_df.head(25),
        p9.aes(x='reorder(feat_name, abs_coef)', y='abs_coef', fill='feat_type'))
    + p9.geom_bar(stat='identity')
    + p9.ylab('Absolute Coefficient')
    + p9.xlab('Features')
    + p9.coords.coord_flip()
    + p9.scale_fill_brewer(type='div', palette=4)
    + p9.theme(figure_size=(3,4.5), legend_position=(0.7,0.35), legend_background=p9.element_blank())
)
fig_rank_1

In [ ]:
feat_coef_df[
    feat_coef_df.apply(lambda x: 'Pool' in x['feat_name'], axis=1)
]

**Stability of the coefficients**

* Run the algorithm multiple times to assess the stability of the model coefficients

In [ ]:
res_stability = []
for i in range(100):
    print(i, end=',', flush=True)
    
    # Use different random seed to run the whole training and prediction process
    res_stability.append(
        train_and_predict(df_normalized, feats, 'logSalePrice', is_log_transformed=True, random_state=1000 + i*100)
    )

In [ ]:
feat_coef_repeated_df = pd.DataFrame()

# Gather the result for multiple runs of the LASSO
for i in range(100):
    current_res = res_stability[i]
    
    temp_df = pd.DataFrame({
            'run_id': i,
            'feat_name': current_res['lasso_cv'].feature_names_in_,
            'coef': current_res['lasso_cv'].coef_,
            'abs_coef': np.abs(current_res['lasso_cv'].coef_)
    })
    
    feat_coef_repeated_df = feat_coef_repeated_df.append(temp_df)

# Aggregate the feature coefficients
feat_coef_summary_df = feat_coef_repeated_df.groupby('feat_name').agg({
    'coef': [np.mean, np.min, np.max],
    'abs_coef': [np.mean, np.min, np.max]
})
feat_coef_summary_df.columns = [
    'coef_mean', 'coef_min', 'coef_max',
    'abs_coef_mean', 'abs_coef_min', 'abs_coef_max']
feat_coef_summary_df.sort_values(by='abs_coef_mean', ascending=False, inplace=True)
feat_coef_summary_df.reset_index(inplace=True)
feat_coef_summary_df['rank'] = np.arange(1, len(feat_coef_summary_df) + 1)

feat_coef_summary_df = add_feat_type(feat_coef_summary_df)

Now, the mean coefficients from multiple run is more stable
* PoolArea ranks # 23

In [ ]:
fig_rank_100 = (
    p9.ggplot(
        feat_coef_summary_df.head(25),
        p9.aes(x='reorder(feat_name, abs_coef_mean)', y='abs_coef_mean', fill='feat_type'))
    + p9.geom_bar(stat='identity')
    + p9.ylab('Absolute Coefficient')
    + p9.xlab('Features')
    + p9.coords.coord_flip()
    + p9.scale_fill_brewer(type='div', palette=4)
    + p9.theme(figure_size=(3,4.5), legend_position=(0.7,0.35), legend_background=p9.element_blank())
)
fig_rank_100

In [ ]:
# Use boxplot to show the spread of the feature coefficients

top_feats = feat_coef_summary_df.head(50)['feat_name']

(
    p9.ggplot(
        feat_coef_repeated_df[feat_coef_repeated_df['feat_name'].isin(top_feats)],
        p9.aes(x='feat_name', y='abs_coef')
    )
    + p9.geom_boxplot()
    + p9.stat_summary(
        fun_y=np.mean, geom="point", size=1, color="red", fill="red")
    + p9.scale_x_discrete(limits=top_feats[-1::-1])
    + p9.coord_flip()
    + p9.ylab('Absolute Coefficients')
    + p9.xlab('Feature Name')
    + p9.theme(figure_size=(6, 10))
)

Create a data frame to contain all feature ranking / coefficients

In [ ]:
feat_coef_df['method'] = 'One Run'
feat_coef_summary_df['method'] = 'Repeated 100 Runs'
feat_coef_both_df = feat_coef_summary_df[[
    'feat_name', 'coef_mean', 'abs_coef_mean', 'feat_type', 'method'
]].head(25).copy().rename(
    columns={'coef_mean': 'coef', 'abs_coef_mean': 'abs_coef'}
)
feat_coef_both_df['feat_name'] = feat_coef_both_df['feat_name'].apply(lambda x: ' ' + x)

feat_coef_both_df = feat_coef_both_df.append(feat_coef_df.head(25))

In [ ]:
(
    p9.ggplot(
        feat_coef_both_df,
        p9.aes(
            x='reorder(feat_name, abs_coef)',
            y='abs_coef',
            fill='feat_type',
            group='method'
        )
    )
    + p9.geom_bar(stat='identity')
    + p9.ylab('Absolute Coefficient')
    + p9.xlab('Features')
    + p9.coords.coord_flip()
    + p9.scale_fill_brewer(type='div', palette=4)
    + p9.facet_wrap('~method', scales='free_y')
    + p9.theme(
        figure_size=(9,5.5),
        legend_position=(1,0.5),
        # legend_background=p9.element_blank(),
        subplots_adjust={'wspace':0.56}
    )
)

**5.4 the Feature Importance**

In [ ]:
# Extract year from "YrSold" one-hot encoded features
YrSold_feat_coef_df = feat_coef_repeated_df[[
    'YrSold' in x for x in feat_coef_repeated_df.feat_name
]]

YrSold_feat_coef_df['year'] = YrSold_feat_coef_df['feat_name'].apply(lambda x: x[-4:])

In [ ]:
# Note: baseline is 2006, which is not encoded as a new feature
(
    p9.ggplot(YrSold_feat_coef_df, p9.aes('year', 'coef', fill='year'))
    + p9.geom_boxplot()
    + p9.geom_hline(yintercept=0, color='r')
    + p9.xlab('Feature Name / Year Sold')
    + p9.ylab('Coefficients')
    # + p9.ggtitle('Feature Coef. for Different YearSold')  # remove title for latex report
    + p9.theme(figure_size=(3, 3), legend_position='none')
)

**Observations:**
* The coef for YrSold_2009 is only slightly lower than the coef for other years. It seems the house market was not heavily impacted by the crisis.

In [ ]:
from datetime import datetime

In [ ]:
df_zillow = pd.read_csv('data/zillow_price_state.csv')

In [ ]:
df_zillow_price_change = (
    df_zillow.loc[:, '2006-01-31':'2011-01-31']  # Only look at the years around the financial crisis
    .apply(lambda x: x/x['2006-01-31'] - 1, axis=1)  # Use 2006-01-31 price as the baseline of calculation
)

In [ ]:
# Combine the price change data back to zillow data, and remove the original price data
df_zillow = pd.concat([df_zillow.iloc[:, :5], df_zillow_price_change], axis=1)

In [ ]:
df_zillow = (
    df_zillow.melt(
        id_vars=['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName'],
        var_name='dt',
        value_name='price_change'
    )
)

df_zillow['dt'] = df_zillow['dt'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [ ]:
(
    p9.ggplot(
        df_zillow[df_zillow['StateName'].isin(['IA', 'CA'])],
        p9.aes('dt', 'price_change', color='StateName')
    )
    + p9.geom_line(size=1)
    + p9.scale_x_datetime(breaks=date_breaks('6 months'), labels=date_format('%b %Y')) 
    + p9.scale_y_continuous(labels=lambda l: ['{:.0%}'.format(x) for x in l])
    + p9.scale_color_brewer(type='qual', palette=2)
    + p9.xlab('Date')
    + p9.ylab('Price Change (Relative to Jan 2006)')
    # + p9.ggtitle('House Price Change Around the Financial Crisis in 2008')
    + p9.theme(
        figure_size=(4, 3),
        axis_text_x=p9.element_text(rotation=30, hjust=1),
        legend_position=(0.75, 0.55),
        legend_background=p9.element_blank()
    )
)